In [1]:
import pandas as pd
import numpy as np
import os
import sys

from synthpop.recipes.starter2 import Starter
from synthpop.census_helpers import Census

# Synthesis flow

![Synthesis](img/hor_synthesis.png)

## 1. What does it mean to synthesize?

`hh` and `p` synthesis datasets are built based on PUMA geographies. The dataset paths shows a `state`, followed by a `county` and an `acs` based year. In the following example, we see synthetic households and persons for the state of Alaska and all the counties that are inside the state. 

The main idea of this process is to use the public user microdata survey (PUMS) which is a sample of the acs answers provided at the maximum dissaggrgated geography as possible (the PUMA) and use it to match each record within a `block group`. Using PUMS and acs subject tables it is possible to build different household types and match persons to them, building a dataset that will show individual records following different characteristics (race of head, tenure, children, etc, etc)  

**Here we have a list of synthesized counties for hh and p from the 02 (AK) state**

In [2]:
ls 02/

02_198.pdf           hh_02_188_2013.csv*  p_02_110_2013.csv*
02_261.pdf           hh_02_195_2013.csv*  p_02_122_2013.csv*
02_290.pdf           hh_02_198_2013.csv*  p_02_130_2013.csv*
hh_02_013_2013.csv*  hh_02_220_2013.csv*  p_02_150_2013.csv*
hh_02_016_2013.csv*  hh_02_230_2013.csv*  p_02_164_2013.csv*
hh_02_020_2013.csv*  hh_02_240_2013.csv*  p_02_170_2013.csv*
hh_02_050_2013.csv*  hh_02_261_2013.csv*  p_02_180_2013.csv*
hh_02_060_2013.csv*  hh_02_270_2013.csv*  p_02_185_2013.csv*
hh_02_068_2013.csv*  hh_02_275_2013.csv*  p_02_188_2013.csv*
hh_02_070_2013.csv*  hh_02_282_2013.csv*  p_02_195_2013.csv*
hh_02_090_2013.csv*  hh_02_290_2013.csv*  p_02_198_2013.csv*
hh_02_100_2013.csv*  p_02_013_2013.csv*   p_02_220_2013.csv*
hh_02_105_2013.csv*  p_02_016_2013.csv*   p_02_230_2013.csv*
hh_02_110_2013.csv*  p_02_020_2013.csv*   p_02_240_2013.csv*
hh_02_122_2013.csv*  p_02_050_2013.csv*   p_02_261_2013.csv*
hh_02_130_2013.csv*  p_02_060_2013.csv*   p_02_270_2013.csv*
hh_02_150_2013.csv*  p_0

In [3]:
# These files looks like...
hh = pd.read_csv('02/hh_02_290_2013.csv')
p = pd.read_csv('02/p_02_290_2013.csv')

In [4]:
hh.head(3)

,Unnamed: 0,serialno,RT,puma00,puma10,NP,TYPE,BLD,TEN,VEH,...,hh_size,hh_workers,seniors,sf_detached,tenure_mover,cat_id,state,county,tract,block group
0,0,2010000910239,H,400,-9,1,1,2.0,1.0,0.0,...,one,one,no,yes,own not recent,3972,2,290,100,1
1,1,2010000165017,H,400,-9,1,1,2.0,1.0,0.0,...,one,one,no,yes,own not recent,3972,2,290,100,1
2,2,2012001118337,H,-9,400,2,1,2.0,2.0,0.0,...,two,one,no,yes,own not recent,4164,2,290,100,1


In [5]:
p.head(3)

,Unnamed: 0,serialno,SPORDER,puma00,puma10,AGEP,JWTR,RELP,SCH,SCHL,...,ESR,HISP,PERNP,RAC1P,hispanic,person_age,person_sex,race,cat_id,hh_id
0,0,2012000016443,1,-9,400,45,1.0,0,1.0,19.0,...,1.0,1,43200.0,4,no,35 to 60,male,other,138262,112
1,1,2012000016443,2,-9,400,45,NaN,1,1.0,21.0,...,6.0,1,0.0,4,no,35 to 60,female,other,138258,112
2,2,2012000016443,4,-9,400,11,NaN,2,2.0,7.0,...,NaN,1,NaN,4,no,19 and under,female,other,138242,112


## 2. What does these tables represent? 

## Starter 

### Creating `marginals` and `joint distributions` for households and persons 

![Starter](img/Starter.png)

### 2.1. The `acs5` subject table 

In [6]:
# census api key
key = os.environ['CENSUS']

# AK state
state = '02'

# 290 county (Yukon-Koyukuk)
county = '290'

In [ ]:
# creates starter object
starter = Starter(key, state, county, acsyear=2013)

In [ ]:
# what do we get from starter class?
print([m for m in dir(starter) if not m.startswith('__')])

This contains a bunch of methods and variables for a synthesized:

In [ ]:
# ...state and county pair
starter.state, starter.county

##### The `Census` object and his methods:  Tract and Block group acs subject tables

![acs](img/subject_table.png)

As it is shown in the diagram, a first step that occurs when we create the `starter` object is that methods from `Census` constructor are also instantiated. This gives us back some relevant information. First, the `acs` 5 years [estimates tables](https://www.census.gov/data/developers/data-sets/acs-5year.html) by different geographies:

In [ ]:
starter.h_acs.head(2)

In [ ]:
# when we instantiate the Starter class to create starter object we are also creating a census object
c = Census(key, acsyear=2013)

In [ ]:
starter.c 

In [ ]:
c

In [ ]:
# that has his own methods 
print([ m for m in dir(c) if not m.startswith('__')])

##### Articulating `Census` and  `Starter` methods

As we said, a first thing that takes place when creating `starter` is the generation of `block groups` and `tracts` tables for a group of variables.

In [ ]:
# we create and merge both tables with:
c.block_group_and_tract_query

In the above diagram it is shown that, inside the `Starter` class some methods from census object `c` are called. But something else to be pointed out here is that, this last one `c` is also calling other methods from `census` module. 

We are interested in the `acs5` method - which is inside the core.py module imported from `import census` module -. Let's check it:

In [ ]:
# imported methods from census module
print([ m for m in dir(c.c) if not m.startswith('__')])

Now, we will define some arbitrary household variables at block group and tract levels to consume `acs5` subject tables.

In [ ]:
# block group
presence_of_children_columns = ['B11005_001E', 'B11005_002E', 'B11005_011E']
block_group_size_attr=['B11005_001E'] # this is the total within the block group

# census tract
vehicle_columns = ['B08201_0%02dE' % i for i in range(1, 7)]
tract_size_attr=['B08201_001E'] # this is the total within the tract

In [ ]:
tr = c.c.acs5.get(['NAME'] + presence_of_children_columns,
                 geo={'for': "tract:*",
                      'in': 'state: 02 county: 290'}, year=2013)

In [ ]:
pd.DataFrame(tr)

In [ ]:
bg = c.c.acs5.get(['NAME'] + presence_of_children_columns,
                  geo={'for': "block group:*",
                       'in': 'state: 02 county: 290'}, year=2013)

In [ ]:
pd.DataFrame(bg)

Both tables are merged at the highest disaggregation level (block group). To do it `scale and merge` Census method scales down from tract to block group level as it is shown below:

In [ ]:
# tract variables
starter.h_acs[['NAME']+vehicle_columns]

Finally, we build the `acs` table for block group and tracts levels - this process is repeated for persons too, here we only demo households subject tables-.

In [ ]:
# the None tract value correspond to all the tracts
h_acs = c.block_group_and_tract_query(block_group_columns=presence_of_children_columns,
                                       tract_columns=vehicle_columns, 
                                       state='02', county='290',
                                       merge_columns=['tract', 'county', 'state'],
                                       block_group_size_attr="B11005_001E",
                                       tract_size_attr="B08201_001E",
                                       tract=None, year=2013)

In [ ]:
h_acs.head(2)

### 2.2. `Categorize` subject tables 

Once we get the subject data for [households](https://github.com/UDST/synthpop/blob/0bc36f8fef036913b416e0d4eb8a3fda79fc70ad/synthpop/recipes/starter2.py#L52-L68) and [persons](https://github.com/UDST/synthpop/blob/0bc36f8fef036913b416e0d4eb8a3fda79fc70ad/synthpop/recipes/starter2.py#L131-L139) variables, the `acs` dataset is categorized by: 

In [ ]:
from synthpop import categorizer as cat

In [ ]:
cat.categorize

This give us back a multindexed table with new names:

In [ ]:
h_acs_cat = cat.categorize(h_acs, {("hh_children", "yes"): "B11005_002E",
                                   ("hh_children", "no"): "B11005_011E",
                                   ("hh_cars", "none"): "B08201_002E",
                                   ("hh_cars", "one"): "B08201_003E",
                                   ("hh_cars", "two or more"):
                                        "B08201_004E + B08201_005E + B08201_006E"},
                           index_cols=['state', 'county', 'tract', 'block group'])

In [ ]:
h_acs_cat

... that can also be obtained by calling it from the `starter` object:

In [ ]:
# here the entire table built in starter2
starter.h_acs_cat

### 2.3. Geography relations for joint distributions

![pums](img/download_pums.png)

As we can see in the diagram, one important method from `Census` constructor is the:

In [ ]:
c.tract_to_puma

... which returns the correspondant puma id for a given tract:

In [ ]:
for tract in ['000100','000200','000300','000400']:
    print('puma10 id for tract {} : {}'.format(tract, c.tract_to_puma(state, county, tract)[0]))
    print('puma00 id for tract {} : {}'.format(tract, c.tract_to_puma(state, county, tract)[1]))
    puma10, puma00 = c.tract_to_puma(state, county, tract)

This information will be mainly used to download `pums` from `aws` or `gcs` (depending on the acs requested year).

##### What we call slices?: the `PUMA` geographies 

![pums_rel](img/pums_tracts.png)

PUMS stands for Public Use Microdata Sample. These are individual records of survey responses with identifying
information removed.These files do not include every record of every person who responded to the ACS. Only a select few that in turn, are representative of the population. 

The ACS samples 3.5 million addresses per year. The 1-year ACS PUMS file contains about 1% of all of the US households. The 5-year ACS PUMS file is the equivalent of five 1-year files, so it includes about 5% of all of the US households.

By contrast, in aggregated tables or summary data, the individual records are categorized and weighted to create an estimate for the larger population. These estimates contains a Margin of Error (which is, to put it into simple words, the percentage of times we do not hit a target population type when we randomly select cases: e.g. 5%, etc. ).

This means that, since `PUMS` microdata provides a sample of the ACS records it is necessary to create an estimate of how many persons/households the raw records represent.  This microdata has no geographies smaller than what we call `PUMAs`.

PUMA is an area where the population of over 100,000 in – population of over 100,000, large enough to meet the disclosure avoidance requirements. It is identified by a five-digit code that is unique within each estate, and they nest within state or state equivalence.

PUMAs are redefined after each Decennial Census. It is important to mention that PUMAs redefined after the 2010 Census were first used in 2012 ACS PUMS files. Multi-year files contain dual PUMAs vintages, for example, the 2010 to 2014 ACS PUMS files. PUMAs are built on Census Tracks and Counties, and can be combined to create rough approximations of towns, counties or cities for analysis.

In [ ]:
# PUMS variables
h_pums_cols = ('serialno', 'PUMA00','PUMA10', 'RT', 'NP', 'TYPE',
               'R65', 'HINCP', 'VEH', 'MV', 'TEN', 'BLD', 'R18')

In [ ]:
h_pums = c.download_household_pums(state, puma10, puma00, usecols=h_pums_cols)

In [ ]:
h_pums

##### Using `PUMS` files

Joint distributions represents a total value for acs queried table. Since this last dataset contains aggregated data for tracts and block groups levels, and given that the PUMS are a representative sample of individual records - each serial number corresponds to a unique answer -, both tables will be used to build target values to be joined during the synthesis. 

In [ ]:
cat.joint_distribution

First, we build a dataframe with the target categories that we obtained querying the `acs subject table`. This, by calling the `category_combinations` method from the categorizer that will return all possible combinations: 

In [ ]:
cat.category_combinations(h_acs_cat.columns)

With this, the `joint_distribution` method will return a sample and categories dataframes. This by following next steps:

In [ ]:
# mapping functions to return values depending on slices dataframes
def cars_cat(r):
    if r.VEH == 0:
        return "none"
    elif r.VEH == 1:
        return "one"
    return "two or more"

def children_cat(r):
    if r.R18 == 1:
        return "yes"
    return "no"

In [ ]:
h_pums, jd_households = cat.joint_distribution(h_pums,
                                                cat.category_combinations(h_acs_cat.columns),
                                                {"hh_cars": cars_cat,
                                                 "hh_children": children_cat,
                                                 })

- **1. Categories dataframe**

This cointains the amount of cases (or frequencies) for each category combination within the PUMA geography. It is important to remark that this totals corresponds to all the tracts we passed to the `tract to puma` function. Given that PUMS returns individual answers for a bunch of variables, we can combine them based on acs subject table columns and get the totals of each combination for a group tracts. This will return a total value for a group of tracts (or PUMA). PUMAs are unique whitin a state.

In [ ]:
jd_households

- **2. Sample dataframe**

This is the microdata puma df we downloaded for tracts 100 to 400, with a new `cat_id` combination column:

In [ ]:
h_pums

**Until this point, we...**:
1. Queried the `acs5` subject table to get persons and households variables at block group and tracts levels
2. Downloaded sample files (PUMS) containing the same variables with different standard names.
3. Matched the tracts of the `county`, `state` pair with a puma id.
4. Built combination of variables at puma level - or all the tracts that are cointained inside a puma id (e.g. households with no children and no cars)

### 2.4. Marginals, the block group level

Marginals are called from the `starter` object inside the `synthesize_all` function and returns...

In [ ]:
#...for a group of available geographies inside a county/state pair
list(starter.get_available_geography_ids())[0]

In [ ]:
indexes = starter.get_available_geography_ids()

In [ ]:
for geog_id in indexes:
    h_marg = starter.get_household_marginal_for_geography(geog_id)

***...the total value of a given variable for the block group geography levels:***

In [ ]:
# This is the marginal table we stored for the last census tract (400)
h_marg

For example, in our acs5 categorized table, in the block group 1 of the 000400 tract (we only stored the last geography since we ran a for loop) there are 82 households with childs and 46 with one car.

In [ ]:
# can check these totals in our acs subject table...
h_acs_cat

In [ ]:
# store the marginal values from the block group 1 of the 000400 census tract
hh_marginals_tract_400_block_gp_1 = h_acs_cat.loc[tuple(list(starter.get_available_geography_ids())[7])]

In [ ]:
hh_marginals_tract_400_block_gp_1

## 3. Sinthesize all: using `Starter` outputs 

### 3.1 The iterative proportional fitting (IPF) procedure 

![constraints](img/constraints.png)

In [ ]:
from synthpop.ipf.ipf import calculate_constraints

In [ ]:
h_constraint, _ = calculate_constraints(hh_marginals_tract_400_block_gp_1, jd_households.frequency)

In [ ]:
# this is our contraints table
h_constraint

In [ ]:
# this is the number of iterations performed to achieve the constraint values
_

In the block group 1 of the census tract 400 we used to have 91 households with no cars, 46 with one car and 39 with two or more:

In [ ]:
bg_targets = hh_marginals_tract_400_block_gp_1[:3]

In [ ]:
bg_targets

At the same time, our `joint_distributions` table gave us the total of that variable for the entire PUMA (in this case, `843 households with no children + 919 households with one children = 1762 households with no cars`:

In [ ]:
jd_households[:2]

In [ ]:
jd_households['frequency'][:2].sum()

With this information, we will build new target values for each category total within the block group by multiplying the target and the proportion that this target represents in the entire PUMA - remember here, that our joint distribution table gets this total by combining different variables. In our case number of cars and children -

`new_target = current_target * (proportion of the current target)`

In [ ]:
sub_category_idx_0 = 0
sub_category_idx_1 = 2

for target in range(len(bg_targets)):
    total_cat = jd_households.frequency[sub_category_idx_0:sub_category_idx_1].sum()
    sub_category_idx_0 += 2
    sub_category_idx_1 += 2
    print('Block group target value: %s'%str(bg_targets[target]))
    print('Total value of the combined category is: %s'%str(total_cat))
    print('Proportion of each category in the combined variables total: %s'%str(bg_targets[target] / total_cat))
    print('New Block group target value: %s'%str(bg_targets[target]*(bg_targets[target] / total_cat)))
    print('*********************************************************************************')

These new values that were built based on the proportion that each block group represents in the PUMA (for every category value) will be used to update our joint distributions table: 

In [ ]:
jd_households.values

...the totals we had for each `category combination`: 0; 1; 2; 3; 4 & 5 will be replaced using that proportion. This way we will define a new maximum value for every category within the block group.

Imagine we have a constraints table with categories from 0 to 2 (has no car and no children, has no car but children and has one car and no children) as we shown in the example above. We will need to:

In [ ]:
current_constraints = jd_households[:3].values.copy()

In [ ]:
# 1. define new targets based on proportions
new_targets = [4.69977298524404, 2.1075697211155378, 1.7706635622817228]

next_constraints = current_constraints.astype(float)

In [ ]:
# 2. update the constraints table
fre = -1
for t in new_targets:
    fre +=1 
    next_constraints[fre][1] = t

In [ ]:
next_constraints

These values will be updated inside an iterative process, where previous and next constraints will be evaluated under a maximum of `tolerance`: 

In [ ]:
tolerance = 1e-3

In [ ]:
np.abs(current_constraints, next_constraints).sum()

In [ ]:
if np.abs(current_constraints, next_constraints).sum()>tolerance:
    print('Recalculate constraints table')

This way, the iteration will continue until reaching a group of new target values under the tolerance defined before. The man result of this process will be...

In [ ]:
# ... our marginals table with total values for the block group
hh_marginals_tract_400_block_gp_1

In [ ]:
# updated with new totals.
h_constraint

In [ ]:
# that will be used to update joint distributions frequencies
jd_households

Consuming the Census API we built a dataset with, for example, 91 aggregated cases for households with no car. Building the constraints table, we used PUMA geographies to determine the proportion that these 91 cases represents in a more dissaggregated level. With this estimation we determine that instead of 91, the block group 1 of the census tract 00400 from the Yukon-Koyukuk county in the Alaska state has 46.5 households with no car. 

### 3.2. The iterative proportional updating (IPU) procedure 

![ipu](img/ipf_ipu.png)

In [ ]:
from synthpop.ipu.ipu import household_weights

In [ ]:
h_constraint.index = jd_households.cat_id

In [ ]:
h_constraint

Besides the `categories dataframe` - our joint distributions table- we mentioned in `2.3`, there is a `sample dataframe` which is the public user microdata sample - our pums.

In [ ]:
households_sample_df = h_pums.copy()

In [ ]:
households_sample_df.head()

In [ ]:
households_sample_df.index.name = "hh_id"

In [ ]:
households_sample_df = households_sample_df.reset_index().set_index("serialno")

In [ ]:
h_freq_table = cat._frequency_table(households_sample_df, jd_households.cat_id)

In [ ]:
h_freq_table = h_freq_table.sort_index(axis=1)

In [ ]:
h_freq_table.head()

Togheter, frequency tables and constraints will be used for weights matrix and fit quality:

In [ ]:
best_weights, fit_quality, iterations = household_weights(h_freq_table,
                                                          None,
                                                          h_constraint,
                                                          None)

In [ ]:
from synthpop.ipu.ipu import _FrequencyAndConstraints

In [ ]:
freq_wrap = _FrequencyAndConstraints(h_freq_table, h_constraint)

This wrapper returns every `cat_id` column with non zero values:

* `0` represents households with no car and no children 
* `1` represents households with no car and children 
* `2` represents households with one car and no children
* `3` represents households with one car and children
* `4` represents households with two or more cars and no children
* `5` represents households with two or more cars and children

The wrapper returns each variables combination (from 0 to 5) inside a tuple with:

1) `cat_id` value

2) a `weights matrix`

3) the new target value (the `constraint` or maximum value that the combination can reach within the block group) we built in the `ipf`.

4) the index of non zero column values 

In [ ]:
# here the information for all the combinations
freq_wrap.iter_columns()

In [ ]:
# and here, we can see for category "0" that non zero values are...
freq_wrap.get_column(0)

In [ ]:
#... in the index 0, 1, 2, 7, (...)
h_freq_table[0][0:8]

With this wrapper, we build the fit quality od each `cat_id`

In [ ]:
from synthpop.ipu.ipu import _average_fit_quality
from synthpop.ipu.ipu import _fit_quality

In [ ]:
# weights matrix
weights = np.ones(len(h_freq_table), dtype='float')

# column (this is the non-zero elements of the "0" cat_id column of the frequency table)
cat_id_0_column = [e for e in freq_wrap.get_column(0)][1]

# nz (this is the idx of the frequency table where non zero values are stored)
cat_id_0_nz = [e for e in freq_wrap.get_column(0)][3]

In [ ]:
# the non zero values should have the same length when filtering the weights matrix
len(cat_id_0_column) == len(weights[cat_id_0_nz])

In [ ]:
# the new target value for the block group
constraint = [e for e in freq_wrap.get_column(0)][2]

In [ ]:
# this is the "fit quality" value for cat_id "0"
_fit_quality(cat_id_0_column, weights[cat_id_0_nz], constraint)

This value is the result of multiplying the non-zero values by the weights matrix, which returns the total frequency of the category within the PUMA: 

In [ ]:
(cat_id_0_column * weights[cat_id_0_nz]).sum()

In [ ]:
# Here the total households with no car and no children(843)
jd_households

The sum of non zero values is weighted and then reduced by substracting the constraints...

In [ ]:
(cat_id_0_column * weights[cat_id_0_nz]).sum() - constraint

... and finally is expressed in terms of the constraint for that category - the 𝛿 parameter described in the IPU paper -. The absolute value of the relative difference between the weighted sum and the corresponding constraint may be used as a goodness of fit measure and is defined as: 

In [ ]:
((cat_id_0_column * weights[cat_id_0_nz]).sum() - constraint) / constraint

This is basically showing the proportion of the `cat_id` within the entire population of the PUMA. In the example of this first iteration, we have that for each household that `cat_id` == `0`, there are 17 households that could be out of that category (`cat_id` != `0`).    

![constraints](img/ipu.png)

This process will be repeated for all household types (or `cat_id`) and build a unique average value (which is the sum of each `_fit_quality` result divided by the number of `cat_id` columns):

In [ ]:
fit_qual_0_to_5 = _average_fit_quality(freq_wrap, weights)

In [ ]:
fit_qual_0_to_5

As it is explained on [A METHODOLOGY TO MATCH DISTRIBUTIONS OF BOTH HOUSEHOLD AND
PERSON ATTRIBUTES IN THE GENERATION OF SYNTHETIC POPULATIONS ](http://www.scag.ca.gov/Documents/PopulationSynthesizerPaper_TRB.pdf), the IPU algorithm starts by assuming equal weights for all households in the sample. The algorithm then proceeds by adjusting weights for each household/person constraint in an iterative process until the constraints are matched as closely as possible for both household and person attributes. 

In [ ]:
fit_change = np.inf
convergence= 1e-4

In [ ]:
if fit_change > convergence:
    print("Updating weights matrix until reaching a fit quality value under the convergence!")

The weights for the each household level constraint are adjusted by dividing the number of households in that category (i.e., the constraint value) by the weighted sum of the first household type column: 

The `_update_weights` creates the following adjustment `adj = constraint / float((column * weights).sum())` and use it to update weights (`weights * adj`). The weights for all households of each household type will be multiplied by this ratio to satisfy the constraint. 

In this sense, the `households_weights` function will finally return a:

In [ ]:
# 1. An array of corrected weights that best matches each household type 
best_weights

In [ ]:
# 2. And a fit quality based on the proportion of each hh type that reduces the fit changes under the convergence 
fit_quality

In [ ]:
# 3. Built in ...
iterations

# 4. Drawing synthetic population 

![draw](img/draw.png)

In [ ]:
from synthpop import draw

In [ ]:
num_households = int(hh_marginals_tract_400_block_gp_1.groupby(level=0).sum().mean())

In [ ]:
num_households

In [ ]:
fac = _FrequencyAndConstraints(h_freq_table, h_constraint)

In [ ]:
indexes = draw._draw_indexes(num_households, fac, best_weights)

In [ ]:
indexes

In [ ]:
synth_hh = h_pums.loc[indexes].reset_index(drop=True)

In [ ]:
synth_hh

In [ ]:
mrg_tbl = pd.DataFrame(
        {'serialno': synth_hh.serialno.values,
         'hh_id': synth_hh.index.values})

In [ ]:
mrg_tbl

Finally, synthetic persons dataset will be built based on `p_pums` and the `hh_id` of synthetic households. As follows: 

```
synth_people = pd.merge(p_pums, mrg_tbl, left_on='serialno', right_on='serialno')
```